In [1]:
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, random_split, DataLoader, Subset
import torch.optim as optim
import torchvision
import torchvision.models as models

import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from torchvision import transforms
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import random
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
def show(image, label):
    plt.imshow(image, cmap='gray')  # Use 'gray' since image is in 'L' mode
    plt.title(f"Label: {label}")
    plt.axis('off')
    plt.show()

In [4]:
df = pd.read_csv('/kaggle/input/processed-lungs/processed_lungs/labels.csv')

nodule_df = df[df['label'] == 1]
second_df = df[df['label'] == 0]

print("we print nodule ")
print(nodule_df)
print("now second one")
print(second_df)

we print nodule 
                   filename  label
0     00000004_000_lung.png      1
1     00000008_002_lung.png      1
2     00000013_025_lung.png      1
3     00000017_000_lung.png      1
4     00000021_000_lung.png      1
...                     ...    ...
6326  00030703_001_lung.png      1
6327  00030715_000_lung.png      1
6328  00030722_000_lung.png      1
6329  00030726_000_lung.png      1
6330  00030793_000_lung.png      1

[6331 rows x 2 columns]
now second one
                     filename  label
6331    00000001_000_lung.png      0
6332    00000001_001_lung.png      0
6333    00000001_002_lung.png      0
6334    00000002_000_lung.png      0
6335    00000003_000_lung.png      0
...                       ...    ...
112111  00030801_001_lung.png      0
112112  00030802_000_lung.png      0
112113  00030803_000_lung.png      0
112114  00030804_000_lung.png      0
112115  00030805_000_lung.png      0

[105785 rows x 2 columns]


In [5]:
BATCH_SIZE = 8
LR = 0.001
EPOCHS = 10

main_dest_dir = '/kaggle/working/'
source_base_dir = '/kaggle/input/processed-lungs'

class ChestXRayDataset(Dataset):
    def __init__(self):
        self.image_paths = []
        self.labels = []
        self.normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                     
        self.transform_positive = transforms.Compose([
            transforms.Resize(299),
            transforms.ToTensor(),
            self.normalize
        ])
        
        self.transform_negative = transforms.Compose([
            transforms.Resize(299),
            transforms.ToTensor(),
            self.normalize
        ])
        
        for cohort_df, label_value in [(nodule_df, 1.0), (second_df, 0.0)]:
            source_folder = 'processed_lungs/nodule' if label_value == 1 else 'processed_lungs/non_nodule'
            for _, row in cohort_df.iterrows():
                image_filename = row['filename']
                img_path = os.path.join(source_base_dir,
                                        source_folder,
                                        image_filename)
        
                self.image_paths.append(img_path)
                self.labels.append(label_value)

    def __getitem__(self, index):
        img_path = self.image_paths[index]
        label = self.labels[index]
        image = Image.open(img_path).convert('RGB')
        if label == 1.0:
            image = self.transform_positive(image)
        else:
            image = self.transform_negative(image)
            
        return image, torch.tensor(label, dtype=torch.float32)

    def __len__(self):
        return len(self.image_paths)
        
    def tackle_idxs(self, idxs):
        image_paths_temp = []
        labels_temp = []
        
        for i in idxs:
            label = self.labels[i]
            img_path = self.image_paths[i]
            
            image_paths_temp.append(img_path)
            labels_temp.append(label)
        
        combined = list(zip(image_paths_temp, labels_temp))
        random.shuffle(combined)
        self.image_paths, self.labels = map(list, zip(*combined))

In [6]:
class Inception_V3(nn.Module):
    def __init__(self):
        super().__init__()
        self.inception_v3 = torchvision.models.inception_v3(pretrained=True)
        
        num_features = self.inception_v3.fc.in_features
        self.inception_v3.fc = nn.Linear(num_features, 1)
        
        if self.inception_v3.aux_logits:
            num_features_aux = self.inception_v3.AuxLogits.fc.in_features
            self.inception_v3.AuxLogits.fc = nn.Linear(num_features_aux, 1)

        for name, param in self.inception_v3.named_parameters():
            param.requires_grad = False
                
        for name, param in self.inception_v3.named_parameters():
            if name.startswith('Mixed_7c') or name.startswith('fc') or name.startswith('AuxLogits.fc'):
                param.requires_grad = True
    
    def forward(self, x):
        return self.inception_v3(x)

In [7]:
dataset_all = ChestXRayDataset()
loader_all = DataLoader(dataset_all, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

model = Inception_V3().to(device)
checkpoint = torch.load("/kaggle/input/inception_v3_final_model_new1-2.pth/pytorch/default/1/Inception_V3_final_model_new1 (2).pth", map_location=device)
model.load_state_dict(checkpoint)
model.eval()

all_labels = []
all_preds = []
all_probs = []

with torch.no_grad():
    for imgs, labels in loader_all:
        imgs = imgs.to(device)
        labels = labels.to(device)
        logits = model(imgs).view(-1)
        probs = torch.sigmoid(logits)
        preds = (probs >= 0.5).float()
        print(preds)
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

accuracy  = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, zero_division=0)
recall    = recall_score(all_labels, all_preds, zero_division=0)
f1        = f1_score(all_labels, all_preds, zero_division=0)
auc       = roc_auc_score(all_labels, all_probs)
fpr, tpr, _ = roc_curve(all_labels, all_probs)

print("===== Full-Dataset Evaluation =====")
print(f"Samples: {len(dataset_all)}")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")
print(f"ROC AUC  : {auc:.4f}")

plt.figure()
plt.plot(fpr, tpr, label=f'ROC AUC = {auc:.4f}')
plt.plot([0,1], [0,1], '--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve — Full Dataset')
plt.legend(loc='lower right')
plt.savefig('Inception_V3_full_dataset_roc.png')
plt.close()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 127MB/s]  
/tmp/ipykernel_31/1521401542.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle 

tensor([1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0')
tensor([1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0')
tensor([1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0')
tensor([1., 1., 1., 1., 0., 1., 1., 1.], device='cuda:0')
tensor([1., 1., 1., 1., 1., 1., 0., 1.], device='cuda:0')
tensor([1., 1., 1., 1., 0., 1., 1., 1.], device='cuda:0')
tensor([0., 0., 1., 0., 1., 1., 1., 1.], device='cuda:0')
tensor([1., 0., 0., 1., 1., 1., 1., 1.], device='cuda:0')
tensor([1., 0., 0., 1., 1., 1., 1., 0.], device='cuda:0')
tensor([1., 1., 1., 1., 1., 0., 0., 1.], device='cuda:0')
tensor([1., 0., 1., 1., 0., 1., 1., 1.], device='cuda:0')
tensor([1., 1., 1., 0., 1., 1., 1., 0.], device='cuda:0')
tensor([1., 0., 1., 1., 0., 1., 1., 1.], device='cuda:0')
tensor([1., 0., 1., 1., 1., 1., 1., 1.], device='cuda:0')
tensor([1., 1., 1., 1., 1., 1., 0., 1.], device='cuda:0')
tensor([0., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0')
tensor([1., 0., 1., 1., 1., 1., 1., 1.], device='cuda:0')
tensor([1., 1.